In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from keras.models import Sequential , Model
from keras.layers import MaxPooling2D , Conv2D , Dense , Activation , Dropout , Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.regularizers import l2, l1

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [8]:
data = pd.read_json('G:/Software/Machine learning/Datasets/Planet json/planesnet.json')
data

,data,labels,locations,scene_ids
0,"[206, 195, 187, 183, 177, 175, 174, 193, 198, ...",1,"[-118.40497658522878, 33.940618514147936]",20170620_175442_0e30
1,"[215, 209, 200, 196, 192, 197, 205, 168, 155, ...",1,"[-122.392469714, 37.6176425378]",20161212_180859_0e30
2,"[204, 214, 220, 219, 213, 205, 198, 193, 199, ...",1,"[-122.397578597, 37.6209247852]",20170524_181349_0e2f
3,"[179, 174, 179, 178, 173, 170, 168, 168, 168, ...",1,"[-122.214849831, 37.7203378331]",20161110_180707_0e1f
4,"[222, 222, 218, 214, 208, 205, 207, 206, 206, ...",1,"[-117.862173435, 33.6796854072]",20160813_184932_0c64
...,...,...,...,...
31995,"[217, 197, 206, 221, 219, 200, 202, 222, 231, ...",0,"[-117.51619965614212, 34.08254781774721]",20170726_174731_0f34
31996,"[184, 198, 218, 222, 220, 216, 216, 215, 210, ...",0,"[-117.2352626288379, 32.79485122236176]",20160726_173906_0e0e
31997,"[121, 121, 119, 117, 119, 112, 118, 117, 126, ...",0,"[-121.4835244137727, 38.53386512064711]",20151030_162249_0c03
31998,"[172, 190, 202, 202, 196, 193, 188, 191, 199, ...",0,"[-118.32580026781089, 33.83093529544055]",20160715_174337_0e0e


In [9]:
data.head()

,data,labels,locations,scene_ids
0,"[206, 195, 187, 183, 177, 175, 174, 193, 198, ...",1,"[-118.40497658522878, 33.940618514147936]",20170620_175442_0e30
1,"[215, 209, 200, 196, 192, 197, 205, 168, 155, ...",1,"[-122.392469714, 37.6176425378]",20161212_180859_0e30
2,"[204, 214, 220, 219, 213, 205, 198, 193, 199, ...",1,"[-122.397578597, 37.6209247852]",20170524_181349_0e2f
3,"[179, 174, 179, 178, 173, 170, 168, 168, 168, ...",1,"[-122.214849831, 37.7203378331]",20161110_180707_0e1f
4,"[222, 222, 218, 214, 208, 205, 207, 206, 206, ...",1,"[-117.862173435, 33.6796854072]",20160813_184932_0c64


In [10]:
data.isnull().sum()

data         0
labels       0
locations    0
scene_ids    0
dtype: int64

In [11]:
data['locations']

0        [-118.40497658522878, 33.940618514147936]
1                  [-122.392469714, 37.6176425378]
2                  [-122.397578597, 37.6209247852]
3                  [-122.214849831, 37.7203378331]
4                  [-117.862173435, 33.6796854072]
                           ...                    
31995     [-117.51619965614212, 34.08254781774721]
31996      [-117.2352626288379, 32.79485122236176]
31997      [-121.4835244137727, 38.53386512064711]
31998     [-118.32580026781089, 33.83093529544055]
31999      [-122.3275363089765, 37.57495862200392]
Name: locations, Length: 32000, dtype: object

### Data Preprocessing

In [14]:
x = []
for d in data['data']:
    d = np.array(d)
    x.append(d.reshape((3 , 20 , 20)).T.reshape((20 , 20 , 3)))
    
x = np.array(x)
y = np.array(data['labels'])

print(x.shape)
print(y.shape)

(32000, 20, 20, 3)
(32000,)


In [17]:
x_train, x_test, y_train, y_test = train_test_split(x , y , test_size = 0.25)

In [18]:
x_train

array([[[[137, 135, 138],
         [143, 141, 143],
         [114, 119, 120],
         ...,
         [140, 129, 114],
         [188, 173, 147],
         [232, 213, 190]],

        [[ 93,  94,  93],
         [114, 114, 116],
         [119, 122, 126],
         ...,
         [132, 124, 120],
         [157, 148, 134],
         [178, 165, 150]],

        [[104, 108, 105],
         [113, 115, 115],
         [121, 124, 129],
         ...,
         [141, 132, 131],
         [147, 140, 143],
         [100,  98,  95]],

        ...,

        [[ 80,  87,  87],
         [ 98, 107, 111],
         [ 92,  96,  99],
         ...,
         [214, 212, 207],
         [198, 198, 194],
         [173, 171, 166]],

        [[117, 121, 118],
         [110, 111, 111],
         [ 97,  99, 104],
         ...,
         [232, 232, 229],
         [227, 227, 226],
         [204, 206, 203]],

        [[170, 172, 167],
         [150, 147, 145],
         [121, 118, 121],
         ...,
         [195, 199, 197],
        

In [22]:
scalar = MinMaxScaler()
scalar.fit(x_train.reshape(x_train.shape[0] , -1))

MinMaxScaler(copy=True, feature_range=(0, 1))

In [23]:
x_train = scalar.transform(x_train.reshape(x_train.shape[0], -1)).reshape(x_train.shape)
x_test = scalar.transform(x_test.reshape(x_test.shape[0], -1)).reshape(x_test.shape)

In [24]:
x_train

array([[[[0.52988048, 0.50617284, 0.53012048],
         [0.552     , 0.54216867, 0.55378486],
         [0.44047619, 0.44715447, 0.46428571],
         ...,
         [0.54183267, 0.48571429, 0.436     ],
         [0.73306773, 0.66393443, 0.56626506],
         [0.90944882, 0.83464567, 0.74409449]],

        [[0.35458167, 0.356     , 0.35458167],
         [0.43824701, 0.43824701, 0.44621514],
         [0.45816733, 0.46586345, 0.484     ],
         ...,
         [0.508     , 0.45867769, 0.46      ],
         [0.61111111, 0.572     , 0.51984127],
         [0.69444444, 0.64285714, 0.58498024]],

        [[0.39840637, 0.4       , 0.40239044],
         [0.432     , 0.41908714, 0.43548387],
         [0.46613546, 0.46311475, 0.48987854],
         ...,
         [0.54581673, 0.49795918, 0.5       ],
         [0.56626506, 0.51882845, 0.54098361],
         [0.38492063, 0.35655738, 0.35483871]],

        ...,

        [[0.30278884, 0.30864198, 0.31983806],
         [0.37698413, 0.39344262, 0.42168675]

### Make CNN model

In [29]:
def make_cnn(data_shape):
    
    kernel_size = 3
    
    model = Sequential()
    
    model.add(Conv2D(16 , (kernel_size) , strides = (1 , 1) , padding = 'valid' , input_shape = (data_shape[1] , data_shape[2] , data_shape[3])))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv2D(32 , (kernel_size) , strides = (1 , 1) , padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv2D(64, (kernel_size), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Conv2D(64, (kernel_size), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2)))

    model.add(Conv2D(64, (kernel_size), strides=(1,1), padding='valid'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2)))
    
    model.add(Flatten())
    
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [30]:
model = make_cnn(x_train.shape)
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 18, 18, 16)        448       
_________________________________________________________________
batch_normalization_5 (Batch (None, 18, 18, 16)        64        
_________________________________________________________________
activation_3 (Activation)    (None, 18, 18, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 32)        4640      
_________________________________________________________________
batch_normalization_6 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
activation_4 (Activation)    (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 64)       

In [31]:
def step_decay(epoch):
    
    initial_lrate = 0.0001
    drop = 0.5
    epochs_drop = 10.0
    
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    
    return lrate

In [32]:
opt = Adam(lr = 0.0001)

lrate = LearningRateScheduler(step_decay)

model.compile(loss='binary_crossentropy', optimizer = opt , metrics = ['accuracy'])

In [36]:
history = model.fit(x_train , y_train , batch_size = 64 , epochs = 15 , shuffle = True , verbose = 2 ,
                                validation_data = (x_test , y_test), callbacks = [lrate],
                               ) 

Train on 24000 samples, validate on 8000 samples
Epoch 1/15
 - 36s - loss: 0.1385 - accuracy: 0.9481 - val_loss: 0.1122 - val_accuracy: 0.9603
Epoch 2/15
 - 36s - loss: 0.1145 - accuracy: 0.9579 - val_loss: 0.0986 - val_accuracy: 0.9649
Epoch 3/15
 - 36s - loss: 0.0975 - accuracy: 0.9631 - val_loss: 0.0851 - val_accuracy: 0.9684
Epoch 4/15
 - 36s - loss: 0.0888 - accuracy: 0.9671 - val_loss: 0.0750 - val_accuracy: 0.9735
Epoch 5/15
 - 36s - loss: 0.0777 - accuracy: 0.9701 - val_loss: 0.1777 - val_accuracy: 0.9334
Epoch 6/15
 - 36s - loss: 0.0716 - accuracy: 0.9737 - val_loss: 0.0644 - val_accuracy: 0.9778
Epoch 7/15
 - 36s - loss: 0.0646 - accuracy: 0.9768 - val_loss: 0.0651 - val_accuracy: 0.9778
Epoch 8/15
 - 36s - loss: 0.0618 - accuracy: 0.9772 - val_loss: 0.0630 - val_accuracy: 0.9774
Epoch 9/15
 - 36s - loss: 0.0551 - accuracy: 0.9801 - val_loss: 0.0587 - val_accuracy: 0.9795
Epoch 10/15
 - 36s - loss: 0.0470 - accuracy: 0.9837 - val_loss: 0.0622 - val_accuracy: 0.9790
Epoch 11/1